In [2]:
from dotenv import load_dotenv
load_dotenv()
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph , START,END
from langgraph.graph.message import add_messages

In [3]:
from langchain.chat_models import init_chat_model

In [4]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from typing import Annotated

# Initialize the model
llm = init_chat_model("google_genai:gemini-2.0-flash")

# Define the state
class State(TypedDict):
    messages: Annotated[list,add_messages]  # keeps track of the full conversation

# Chatbot function
def chatbot(state: State) -> State:
    response = llm.invoke(state["messages"])
    return {"messages": state["messages"] + [response]}

# Build the graph
builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()


In [5]:
message = {"role":"user","content":"who walked on the moon for the first time? print only the name"}
response=graph.invoke({"messages":[message]})
response["messages"]

[HumanMessage(content='who walked on the moon for the first time? print only the name', additional_kwargs={}, response_metadata={}, id='1ffddff5-a026-4aa9-9e00-c0d6ae854f16'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f7f19e10-edcb-46ce-ac68-c9a6e379fcf4-0', usage_metadata={'input_tokens': 14, 'output_tokens': 3, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]

In [7]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit","exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

Bot: Rakesh Sharma is a retired Indian Air Force officer and cosmonaut who became the first Indian citizen to travel to space. He flew aboard the Soviet Soyuz T-11 in 1984, spending nearly eight days on the Salyut 7 space station. He is considered a national hero in India.
Bot: Rakesh Sharma was born on **January 13, 1949**.


KeyboardInterrupt: Interrupted by user